In [ ]:
import numpy as np 
import pandas as pd
import keras

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
housing = pd.read_csv("/kaggle/input/real-estate-dataset/data.csv")

housing.describe()

In [ ]:
housing = housing.dropna(axis='rows')

In [ ]:
housing.isnull().sum()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt 
housing.hist(bins=50, figsize=(20,15)) 
plt.show()

In [ ]:
data = housing.iloc[:,0:12]
y = housing.iloc[:,13]

In [ ]:
from sklearn.model_selection import train_test_split
x, x_test, y, y_test = train_test_split(data,y,test_size=0.2,train_size=0.8)
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size = 0.25,train_size =0.75)

In [ ]:
corr_matrix = housing.corr()

In [ ]:
corr_matrix["MEDV"].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ["MEDV", "RM", "ZN", "B"]
scatter_matrix(housing[attributes], figsize=(12, 8))

In [ ]:
housing.plot(kind="scatter", x="RM", y="MEDV", alpha=0.1)

In [ ]:
data = housing[["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B","LSTAT"]] 
target = housing["MEDV"] 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

In [ ]:
def build_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(64, activation="relu", input_shape=x_train.shape[1:])) 
    model.add(keras.layers.Dense(64, activation="relu"))  
    model.add(keras.layers.Dense(1))
    model.compile(loss="mse", metrics = ['mae'], optimizer="rmsprop")
    print(model.summary())
    return model


In [ ]:
model = build_model()
history = model.fit(x_train_scaled, y_train, epochs=100, validation_data=(x_val_scaled, y_val), verbose=0)
val_mse, val_mae = model.evaluate(x_val_scaled, y_val, verbose=0)

print(val_mse, val_mae)

In [ ]:
plt.plot(history.history['loss'], label='MAE (training data)')
plt.plot(history.history['val_loss'], label='MAE (validation data)')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
x_new = x_test_scaled[:3] # pretend these are new instances
y_pred = model.predict(x_new)
y_pred